<center>
    <h1> INF285 - Computación Científica </h1>
    <h2> Tarea 4 - Código Base</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> Version: 1.00</h2>
</center>

# No debe utilizar bibliotecas adicionales.

In [231]:
import numpy as np
from scipy import linalg

# Funciones a implementar:

In [237]:
# N = n <<<<<<<<<<<<<<<<<<<<<<<< ojo
def matrix_A(alfa, n):
    """
    Parameters
    ----------
    alfa          :float
                     Thermal conductivity
    n             : int
                     Number of coefficients
    Returns
    -------
    A             : (n,n)-array
                     Heat coefficients
    """
    #nxn matriz
    delta_x = 1/(n+1)
    A = (alfa/(delta_x**2))*(np.diag(np.full(n,-2))+np.diag(np.ones(n-1),1)+np.diag(np.ones(n-1),-1))
    return A

# El código de GMRes lo obtuve del jupyter notebook 09_GMRes.ipynb 
def GMREs(A,b,x0=np.array([0.0]), m=10, threshold=1e-12):
    """
    Parameters
    ----------
    A              : (n,n)-array 
                    Discretization matrix
    b              : (n, )-array
                    Target               
    x0             : (n,)-array float
                    Initial conditions array
    m              : int
                    Krylov space Components
    threshold      : float
                    Minimum residue error value           

    Returns
    -------
    x             : (n, )-array
                     Solutions
    out_res       : (n,)-array
                     Residual error
    """
    n = len(b)
    if len(x0)==1:
        x0=np.zeros(n)
    r0 = b - np.dot(A, x0)
    nr0=np.linalg.norm(r0)
    out_res=np.array(nr0)
    Q = np.zeros((n,n))
    H = np.zeros((n,n))
    Q[:,0] = r0 / nr0
    flag_break=False
    for k in np.arange(np.min((m,n))):
        y = np.dot(A, Q[:,k])
        for j in np.arange(k+1):
            H[j][k] = np.dot(Q[:,j], y)
            y = y - np.dot(H[j][k],Q[:,j]) 
        if k+1<n:
            H[k+1][k] = np.linalg.norm(y)
            if (np.abs(H[k+1][k]) > 1e-16):
                Q[:,k+1] = y/H[k+1][k]
            else:
                flag_break=True
            e1 = np.zeros((k+1)+1)        
            e1[0]=1
            H_tilde=H[0:(k+1)+1,0:k+1]
        else:
            H_tilde=H[0:k+1,0:k+1]
        ck = np.linalg.lstsq(H_tilde, nr0*e1,rcond=None)[0] 
        if k+1<n:
            x = x0 + np.dot(Q[:,0:(k+1)], ck)
        else:
            x = x0 + np.dot(Q, ck)
        norm_small=np.linalg.norm(np.dot(H_tilde,ck)-nr0*e1)
        out_res = np.append(out_res,norm_small)
        if flag_break:
            break
        if norm_small<threshold:
            break
    return x, out_res

def eig_vals_and_vects(A, iv):
    """
    Parameters
    ----------
    A              : (n,n)-array 
                    Discretization matrix
    iv             : (n,)-array float
                    Initial values array         
    Returns
    -------
    eVals          : (n, )-array
                     Eigenvalues
    eVects         : (n,n)-array
                     Eigenvectors
    C              : (n, )-array
                     System solution
    residue        : (n, )-array
                     GMREs residue              
    """
    n,_ = A.shape
    #Valores y vectores propios de A
    eVals, eVects = linalg.eig(A)
    eVals = eVals.real
    #Construir matriz M
    M = np.zeros((n,n)) 
    t = 0
    for i in range(n):
      #M[:,i]= np.exp(eVals[i]*t)*eVects[:,i]
      M[:,i]= np.exp(eVals[i]*t)*eVects[:,i]
    #Calcular GMRes para C
    C, residue = GMREs(M,iv,x0=np.array([0.0]), m=10, threshold=1e-12)
    return eVals, eVects, C, residue


def build_u(n, k, iv):
    """
    Parameters
    ----------
    n             : int
                   Discretization components
    k             : int
                   thermal diffusivity                    
    iv            : (n, )-array float
                   initial values array
    Returns
    -------
    u             : callable
                   Solved heat function         
    """
    
    def fun(t,eVals, eVects, C, residue):
      U_aux = 0
      for i in range(n): 
        U_aux += C[i]*np.exp(eVals.real[i]*t)*eVects[:,i]
      return U_aux
    
    #Contruir matriz A
    A = matrix_A(k, n)
    # Obtener valores y vect propios de A y resolver para C el sistema M*C=iv en t=0
    eVals, eVects, C, residue = eig_vals_and_vects(A, iv)
    # contruir u para t!=0
    u = lambda t: fun(t,eVals, eVects, C, residue)
    return u



In [235]:
# puede hacer pruebas con el siguiente iv = np.array([0,0.707107, 1.0, 0.707107,0]) con n=5

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a11cc422-5899-4dee-a9b0-08ff225abaf8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>